# GoogleMapsScraper Script


### Objective:
Scraper of Google Maps reviews. The code allows to extract the most recent reviews starting from the url of a specific Point Of Interest (POI) in Google Maps. An additional extension helps to monitor and incrementally store the reviews in a MongoDB instance.

In [1]:
import itertools
import logging
import re
import time
import traceback
from datetime import datetime
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ChromeOptions as Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

GM_WEBPAGE = 'https://www.google.com/maps/'
MAX_WAIT = 10
MAX_RETRY = 5
MAX_SCROLLS = 40

class GoogleMapsScraper:

    def __init__(self, debug=False):
        self.debug = debug
        self.driver = self.__get_driver()
        self.logger = self.__get_logger()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, tb):
        if exc_type is not None:
            traceback.print_exception(exc_type, exc_value, tb)

        self.driver.close()
        self.driver.quit()

        return True

    def sort_by(self, url, ind):

        self.driver.get(url)
        self.__click_on_cookie_agreement()

        wait = WebDriverWait(self.driver, MAX_WAIT)

        # open dropdown menu
        clicked = False
        tries = 0
        while not clicked and tries < MAX_RETRY:
            try:
                menu_bt = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@data-value=\'Sort\']')))
                menu_bt.click()

                clicked = True
                time.sleep(3)
            except Exception as e:
                tries += 1
                self.logger.warning('Failed to click sorting button')

            # failed to open the dropdown
            if tries == MAX_RETRY:
                return -1

        #  element of the list specified according to ind
        recent_rating_bt = self.driver.find_elements(By.XPATH, '//div[@role=\'menuitemradio\']')[ind]
        recent_rating_bt.click()

        # wait to load review (ajax call)
        time.sleep(5)

        return 0

    def get_places(self, keyword_list=None):

        df_places = pd.DataFrame()
        search_point_url_list = self._gen_search_points_from_square(keyword_list=keyword_list)

        for i, search_point_url in enumerate(search_point_url_list):
            print(search_point_url)

            if (i+1) % 10 == 0:
                print(f"{i}/{len(search_point_url_list)}")
                df_places = df_places[['search_point_url', 'href', 'name', 'rating', 'num_reviews', 'close_time', 'other']]
                df_places.to_csv('output/places_wax.csv', index=False)


            try:
                self.driver.get(search_point_url)
            except NoSuchElementException:
                self.driver.quit()
                self.driver = self.__get_driver()
                self.driver.get(search_point_url)

            # scroll to load all (20) places into the page
            scrollable_div = self.driver.find_element(By.CSS_SELECTOR,
                "div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div[aria-label*='Results for']")
            for i in range(10):
                self.driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)

            # Get places names and href
            time.sleep(2)
            response = BeautifulSoup(self.driver.page_source, 'html.parser')
            div_places = response.select('div[jsaction] > a[href]')

            for div_place in div_places:
                place_info = {
                    'search_point_url': search_point_url.replace('https://www.google.com/maps/search/', ''),
                    'href': div_place['href'],
                    'name': div_place['aria-label']
                }

                df_places = df_places.append(place_info, ignore_index=True)

            # TODO: implement click to handle > 20 places

        df_places = df_places[['search_point_url', 'href', 'name']]
        df_places.to_csv('output/places_wax.csv', index=False)



    def get_reviews(self, offset):

        # scroll to load reviews
        self.__scroll()

        # wait for other reviews to load (ajax)
        time.sleep(4)

        # expand review text
        self.__expand_reviews()

        # parse reviews
        response = BeautifulSoup(self.driver.page_source, 'html.parser')
        # TODO: Subject to changes
        rblock = response.find_all('div', class_='jftiEf fontBodyMedium')
        parsed_reviews = []
        for index, review in enumerate(rblock):
            if index >= offset:
                r = self.__parse(review)
                parsed_reviews.append(r)

                # logging to std out
                print(r)

        return parsed_reviews



    # need to use different url wrt reviews one to have all info
    def get_account(self, url):

        self.driver.get(url)
        self.__click_on_cookie_agreement()

        # ajax call also for this section
        time.sleep(2)

        resp = BeautifulSoup(self.driver.page_source, 'html.parser')

        place_data = self.__parse_place(resp, url)

        return place_data


    def __parse(self, review):

        item = {}

        try:
            # TODO: Subject to changes
            id_review = review['data-review-id']
        except Exception as e:
            id_review = None

        try:
            # TODO: Subject to changes
            username = review['aria-label']
        except Exception as e:
            username = None

        try:
            # TODO: Subject to changes
            review_text = self.__filter_string(review.find('span', class_='wiI7pd').text)
        except Exception as e:
            review_text = None

        try:
            # TODO: Subject to changes
            rating = float(review.find('span', class_='kvMYJc')['aria-label'].split(' ')[0])
        except Exception as e:
            rating = None

        try:
            # TODO: Subject to changes
            relative_date = review.find('span', class_='rsqaWe').text
        except Exception as e:
            relative_date = None

        try:
            n_reviews = review.find('div', class_='RfnDt').text.split(' ')[3]
        except Exception as e:
            n_reviews = 0

        try:
            user_url = review.find('button', class_='WEBjve')['data-href']
        except Exception as e:
            user_url = None

        item['id_review'] = id_review
        item['caption'] = review_text

        # depends on language, which depends on geolocation defined by Google Maps
        # custom mapping to transform into date should be implemented
        item['relative_date'] = relative_date

        # store datetime of scraping and apply further processing to calculate
        # correct date as retrieval_date - time(relative_date)
        item['retrieval_date'] = datetime.now()
        item['rating'] = rating
        item['username'] = username
        item['n_review_user'] = n_reviews
        #item['n_photo_user'] = n_photos  ## not available anymore
        item['url_user'] = user_url

        return item


    def __parse_place(self, response, url):

        place = {}

        try:
            place['name'] = response.find('h1', class_='DUwDvf fontHeadlineLarge').text.strip()
        except Exception as e:
            place['name'] = None

        try:
            place['overall_rating'] = float(response.find('div', class_='F7nice ').find('span', class_='ceNzKf')['aria-label'].split(' ')[1])
        except Exception as e:
            place['overall_rating'] = None

        try:
            place['n_reviews'] = int(response.find('div', class_='F7nice ').text.split('(')[1].replace(',', '').replace(')', ''))
        except Exception as e:
            place['n_reviews'] = 0

        try:
            place['n_photos'] = int(response.find('div', class_='YkuOqf').text.replace('.', '').replace(',','').split(' ')[0])
        except Exception as e:
            place['n_photos'] = 0

        try:
            place['category'] = response.find('button', jsaction='pane.rating.category').text.strip()
        except Exception as e:
            place['category'] = None

        try:
            place['description'] = response.find('div', class_='PYvSYb').text.strip()
        except Exception as e:
            place['description'] = None

        b_list = response.find_all('div', class_='Io6YTe fontBodyMedium')
        try:
            place['address'] = b_list[0].text
        except Exception as e:
            place['address'] = None

        try:
            place['website'] = b_list[1].text
        except Exception as e:
            place['website'] = None

        try:
            place['phone_number'] = b_list[2].text
        except Exception as e:
            place['phone_number'] = None
    
        try:
            place['plus_code'] = b_list[3].text
        except Exception as e:
            place['plus_code'] = None

        try:
            place['opening_hours'] = response.find('div', class_='t39EBf GUrTXd')['aria-label'].replace('\u202f', ' ')
        except:
            place['opening_hours'] = None

        place['url'] = url

        lat, long, z = url.split('/')[6].split(',')
        place['lat'] = lat[1:]
        place['long'] = long

        return place


    def _gen_search_points_from_square(self, keyword_list=None):
        # TODO: Generate search points from corners of square

        keyword_list = [] if keyword_list is None else keyword_list

        square_points = pd.read_csv('input/square_points.csv')

        cities = square_points['city'].unique()

        search_urls = []

        for city in cities:

            df_aux = square_points[square_points['city'] == city]
            latitudes = df_aux['latitude'].unique()
            longitudes = df_aux['longitude'].unique()
            coordinates_list = list(itertools.product(latitudes, longitudes, keyword_list))

            search_urls += [f"https://www.google.com/maps/search/{coordinates[2]}/@{str(coordinates[1])},{str(coordinates[0])},{str(15)}z"
             for coordinates in coordinates_list]

        return search_urls


    # expand review description
    def __expand_reviews(self):
        # use XPath to load complete reviews
        # TODO: Subject to changes
        buttons = self.driver.find_elements(By.CSS_SELECTOR,'button.w8nwRe.kyuRq')
        for button in buttons:
            self.driver.execute_script("arguments[0].click();", button)


    def __scroll(self):
        # TODO: Subject to changes
        scrollable_div = self.driver.find_element(By.CSS_SELECTOR,'div.m6QErb.DxyBCb.kA9KIf.dS8AEf')
        self.driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        #self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


    def __get_logger(self):
        # create logger
        logger = logging.getLogger('googlemaps-scraper')
        logger.setLevel(logging.DEBUG)

        # create console handler and set level to debug
        fh = logging.FileHandler('gm-scraper.log')
        fh.setLevel(logging.DEBUG)

        # create formatter
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

        # add formatter to ch
        fh.setFormatter(formatter)

        # add ch to logger
        logger.addHandler(fh)

        return logger


    def __get_driver(self, debug=False):
        options = Options()

        if not self.debug:
            options.add_argument("--headless")
        else:
            options.add_argument("--window-size=1366,768")

        options.add_argument("--disable-notifications")
        #options.add_argument("--lang=en-GB")
        options.add_argument("--accept-lang=en-GB")
        input_driver = webdriver.Chrome(service=Service(), options=options)

         # click on google agree button so we can continue (not needed anymore)
         # EC.element_to_be_clickable((By.XPATH, '//span[contains(text(), "I agree")]')))
        input_driver.get(GM_WEBPAGE)

        return input_driver

    # cookies agreement click
    def __click_on_cookie_agreement(self):
        try:
            agree = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//span[contains(text(), "Reject all")]')))
            agree.click()

            # back to the main page
            # self.driver.switch_to_default_content()

            return True
        except:
            return False

    # util function to clean special characters
    def __filter_string(self, str):
        strOut = str.replace('\r', ' ').replace('\n', ' ').replace('\t', ' ')
        return strOut


# Basic Usage
The Google_Review_Scraper.py script needs two main parameters as input:

--i: input file name, containing a list of urls that point to Google Maps place reviews (default: urls.txt)
--N: number of reviews to retrieve, starting from the most recent (default: 100)
Example:

python scraper.py --N 50

generates a csv file containing last 50 reviews of places present in urls.txt

In current implementation, the CSV file is handled as an external function, so if you want to change path and/or name of output file, you need to modify that function.

### Additionally, other parameters can be provided:

--place: boolean value that allows to scrape POI metadata instead of reviews (default: false)
--debug: boolean value that allows to run the browser using the graphical interface (default: false)
--source: boolean value that allows to store source URL as additional field in CSV (default: false)
--sort-by: string value among most_relevant, newest, highest_rating or lowest_rating (default: newest), developed by @quaesito and that allows to change sorting behavior of reviews

In [2]:
from datetime import datetime, timedelta
from termcolor import colored
import argparse
import time
import sys
import csv

# Sorting index mapping
ind = {'most_relevant': 0, 'newest': 1, 'highest_rating': 2, 'lowest_rating': 3}
HEADER = ['id_review', 'caption', 'relative_date', 'retrieval_date', 'rating', 'username', 'n_review_user', 'n_photo_user', 'url_user']
HEADER_W_SOURCE = ['id_review', 'caption', 'relative_date', 'retrieval_date', 'rating', 'username', 'n_review_user', 'n_photo_user', 'url_user', 'url_source']


def csv_writer(source_field, ind_sort_by, path='D:\\MTech\\4-SEM\\ProjectCode\\'):
    outfile= ind_sort_by + '_gm_reviews.csv'
    targetfile = open(path + outfile, mode='w', encoding='utf-8', newline='\n')
    writer = csv.writer(targetfile, quoting=csv.QUOTE_MINIMAL)

    if source_field:
        h = HEADER_W_SOURCE
    else:
        h = HEADER
    writer.writerow(h)

    return writer


if __name__ == '__main__':
    # Filter out unwanted arguments
    argv = sys.argv
    args_to_parse = [arg for arg in argv if arg.startswith('--')]

    parser = argparse.ArgumentParser(description='Google Maps reviews scraper.')
    parser.add_argument('--N', type=int, default=1000, help='Number of reviews to scrape')
    parser.add_argument('--i', type=str, default='D:\\MTech\\4-SEM\\ProjectCode\\URLS.txt', help='Target URLs file')
    parser.add_argument('--sort_by', type=str, default='newest', help='Sorting order: most_relevant, newest, highest_rating, or lowest_rating')
    parser.add_argument('--place', dest='place', action='store_true', help='Scrape place metadata')
    parser.add_argument('--debug', dest='debug', action='store_true', help='Run scraper using browser graphical interface')
    parser.add_argument('--source', dest='source', action='store_true', help='Add source URL to CSV file (for multiple URLs in a single file)')
    parser.set_defaults(place=False, debug=False, source=False)

    args = parser.parse_args(args_to_parse)
    
    # store reviews in CSV file
    writer = csv_writer(args.source, args.sort_by)
    
    with GoogleMapsScraper(debug=args.debug) as scraper:
        with open(args.i, 'r') as urls_file:
            for url in urls_file:
                if args.place:
                    print(scraper.get_account(url))
                else:
                    error = scraper.sort_by(url, ind[args.sort_by])

                    if error == 0:

                        n = 0

                        #if ind[args.sort_by] == 0:
                        #    scraper.more_reviews()

                        while n < args.N:

                            # logging to std out
                            print(colored('[Review ' + str(n) + ']', 'cyan'))

                            reviews = scraper.get_reviews(n)
                            if len(reviews) == 0:
                                break

                            for r in reviews:
                                row_data = list(r.values())
                                if args.source:
                                    row_data.append(url[:-1])

                                writer.writerow(row_data)

                            n += len(reviews)

    

[Review 0]
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSURIcFpHY2dnRRAB', 'caption': 'One of the most beautiful institutes in India. Almost every area is photogenic. Very clean and very well organised. Also the food of the stalls and the mess here is great and has a huge variety. My brother was studying here and had one of the best times in his life here. ⭐⭐⭐⭐', 'relative_date': '23 hours ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 26, 25, 381633), 'rating': 4.0, 'username': 'ShortSpot', 'n_review_user': '21', 'url_user': 'https://www.google.com/maps/contrib/104034199958290567626/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSURIbWJqbkxnEAE', 'caption': '10% attendance component bad', 'relative_date': 'a day ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 26, 25, 382635), 'rating': 3.0, 'username': 'Aayush Patra', 'n_review_user': '30', 'url_user': 'https://www.google.com/maps/contrib/109988514290727449178/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUNIMUx

{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUR6aEs2OW1nRRAB', 'caption': None, 'relative_date': '3 months ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 26, 33, 846404), 'rating': 5.0, 'username': 'Creative ideas', 'n_review_user': '166', 'url_user': 'https://www.google.com/maps/contrib/113969449616277482574/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUN6bWVucUFREAE', 'caption': None, 'relative_date': '3 months ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 26, 33, 847762), 'rating': 5.0, 'username': 'Srinivas Goud', 'n_review_user': '51', 'url_user': 'https://www.google.com/maps/contrib/113558480576484727882/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUN6MUlUWVVBEAE', 'caption': None, 'relative_date': '3 months ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 26, 33, 847762), 'rating': 5.0, 'username': 'Kotagiri Chiranjeevi Chary', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/105709765070004385859/reviews?hl=en'}
{'id_r

{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUMxOHVmZm9nRRAB', 'caption': 'Very clean and well maintained', 'relative_date': '8 months ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 26, 46, 574160), 'rating': 5.0, 'username': 'Pradeep M', 'n_review_user': '174', 'url_user': 'https://www.google.com/maps/contrib/103698798426451976652/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSURWN195Tzh3RRAB', 'caption': None, 'relative_date': '9 months ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 26, 46, 575171), 'rating': 5.0, 'username': 'Bhargav Ponnam', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/107929370269007730148/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSURsb1BtQ2FnEAE', 'caption': None, 'relative_date': '9 months ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 26, 46, 575171), 'rating': 5.0, 'username': 'Ramesh Naidu', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/105320833307241643104/reviews?hl=en'

{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUNKOXQyeC1RRRAB', 'caption': None, 'relative_date': 'a year ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 26, 59, 785722), 'rating': 5.0, 'username': 'Praveen Babu', 'n_review_user': '3', 'url_user': 'https://www.google.com/maps/contrib/104948558309324786343/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUNKaXNuNUd3EAE', 'caption': None, 'relative_date': 'a year ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 26, 59, 785722), 'rating': 5.0, 'username': 'Pranay Singh', 'n_review_user': '45', 'url_user': 'https://www.google.com/maps/contrib/110178661281501501233/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUNKdE1pVG1RRRAB', 'caption': 'Best institute for learning', 'relative_date': 'a year ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 26, 59, 786722), 'rating': 5.0, 'username': 'shaik omer', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/106322735922482571515/reviews?hl=en'}
{'id_re

{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUN4eTRfMVZnEAE', 'caption': 'Awsome', 'relative_date': 'a year ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 27, 8, 574215), 'rating': 5.0, 'username': 'Bhaskar Samanthapudi', 'n_review_user': '71', 'url_user': 'https://www.google.com/maps/contrib/107840611502332971554/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUN4eTlmb1l3EAE', 'caption': None, 'relative_date': 'a year ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 27, 8, 575215), 'rating': 5.0, 'username': 'Krishna Kishore', 'n_review_user': '10', 'url_user': 'https://www.google.com/maps/contrib/110852887351365822745/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUN4clllZ2xBRRAB', 'caption': None, 'relative_date': 'a year ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 27, 8, 575215), 'rating': 4.0, 'username': 'Pradip Mondal', 'n_review_user': '67', 'url_user': 'https://www.google.com/maps/contrib/108138670846422068296/reviews?hl=en'}
{'id_review': 

...

{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUR5NDlYOTBnRRAB', 'caption': None, 'relative_date': '3 years ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 33, 34, 186303), 'rating': 5.0, 'username': 'Pavan Giram', 'n_review_user': '1', 'url_user': 'https://www.google.com/maps/contrib/109015817954636186219/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUR5dzZyR1Z3EAE', 'caption': None, 'relative_date': '3 years ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 33, 34, 186303), 'rating': 5.0, 'username': 'Mohd Afroz', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/104168777473340450001/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUR5X2RIbTJRRRAB', 'caption': None, 'relative_date': '3 years ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 33, 34, 186303), 'rating': 5.0, 'username': 'Mujib Shaikh', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/116362100471992974791/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0

{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUN5NzhfOFJ3EAE', 'caption': 'Ok', 'relative_date': '3 years ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 33, 52, 205772), 'rating': 3.0, 'username': 'Sathish Kattekolu', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/110492148411899304864/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUN5NzdQV0R3EAE', 'caption': None, 'relative_date': '3 years ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 33, 52, 206772), 'rating': 5.0, 'username': 'Dilip Mamilla', 'n_review_user': '11', 'url_user': 'https://www.google.com/maps/contrib/101579143346550558374/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUN5ajlpWGRBEAE', 'caption': 'I am one of those lucky people who had a chance to stay inside this beautiful and prestigious campus. This is a good place for budding students looking forward to learn independently. They have an amazing infrastructure. The best part was the view. You can see peacocks every time. 

{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUN5bEtEZ1ZBEAE', 'caption': None, 'relative_date': '3 years ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 34, 10, 306361), 'rating': 5.0, 'username': 'Nagapuri Sagargoud', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/111848056434366791448/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUN5NVAtRDJRRRAB', 'caption': None, 'relative_date': '3 years ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 34, 10, 307367), 'rating': 5.0, 'username': 'shaik Ahmed', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/107046453372787013202/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUN5NUphSUpnEAE', 'caption': None, 'relative_date': '3 years ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 34, 10, 307367), 'rating': 5.0, 'username': 'Persis Blessena', 'n_review_user': '6', 'url_user': 'https://www.google.com/maps/contrib/118306204101069285391/reviews?hl=en'}
{'id_review': 'ChZDSUhN

{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSURTbFlpeVhBEAE', 'caption': None, 'relative_date': '3 years ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 34, 29, 421175), 'rating': 5.0, 'username': 'Raghu Ram C S', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/117644026929748688906/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSURTaWJEUnhnRRAB', 'caption': None, 'relative_date': '3 years ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 34, 29, 422177), 'rating': 5.0, 'username': 'Gajapathi Rajasekhar', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/116720295880331166216/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSURzN091R3lnRRAB', 'caption': None, 'relative_date': '3 years ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 34, 29, 422177), 'rating': 5.0, 'username': 'Bhavani Shankar Balla', 'n_review_user': '38', 'url_user': 'https://www.google.com/maps/contrib/113317361179297198766/reviews?hl=en'}
{'id_review

{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUNTb3N1a2NREAE', 'caption': None, 'relative_date': '3 years ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 34, 48, 473939), 'rating': 2.0, 'username': 'Mohammed Abdul Saleem', 'n_review_user': '4', 'url_user': 'https://www.google.com/maps/contrib/108461483209585551539/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUNTb3QzSVhnEAE', 'caption': None, 'relative_date': '3 years ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 34, 48, 473939), 'rating': 5.0, 'username': 'rudra bal', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/113912145434799950402/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUNTNEl1eXJBRRAB', 'caption': None, 'relative_date': '3 years ago', 'retrieval_date': datetime.datetime(2024, 9, 19, 13, 34, 48, 473939), 'rating': 5.0, 'username': 'Srinivas Gajjala', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/109689235970425963770/reviews?hl=en'}
{'id_review': 'ChZDSU